```{image} https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png 
:width: 140px
:align: left
```
## Image Processing Laboratory Notebooks
---

This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-511 Image Processing I**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2023.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Yan Liu](mailto:yan.liu@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    
---

# Lab 2.1: Filtering
**Released**: Thursday November 9, 2023

**Submission**: Monday November 20, 2023 before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=522)

**Grade weight**: Lab 2 (16 points), 10% of the overall grade

**Related lectures**: Chapter 3

Double-click on this cell, fill your name and SCIPER number below to verify your identity in Noto and set the seed for random results.
:::{attention} Please write down your name and SCIPER! 
### Student Name: 
### SCIPER: 
:::

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

(imports)=
### Imports
Run the next two cell to import the libraries and load the images that we will use throughout the lab.

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
import scipy.ndimage as ndi
import ipywidgets as widgets
from interactive_kit import imviewer as viewer

# Load images to be used in this exercise 
bikesgray = io.imread('images/bikesgray.tif').astype('float64')
camera = io.imread('images/camera-16bits.tif').astype('float64')
spots = io.imread('images/spots.tif').astype('float64')

In [ ]:
%use javascript
%get bikesgray camera spots
// import ImageAccess as Image
var Image = require('image-access')

# Filtering (9 points)

In this lab you will review the basics of digital image filtering: how to implement a convolution (the main ingredient of filtering), separable and nonseparable filters, filter design and boundary conditions. The goal is to fully understand the filtering algorithm at a pixel-by-pixel level by implementing it in JavaScript, and at the same time to learn the standard tools to use filtering in professional image-processing libraries in Python.

(ToC_1_Filtering)=
# Table of contents
1. [Edge detection](#edge-detection)
    1. [Separable implementation](#separable-implementation) (**2 points**)
    2. [Nonseparable implementation](#non-separable-implementation) (**1 point**)
    3. [Comparison: Separable vs Nonseparable](#comparison-separable-vs-non-separable) 
    4. [Edge detection in Python](#edge-detection-in-python)
2. [Implementation and classification of digital filters](#implementation-and-classification-of-digital-filters) (**6 points**)
    1. [Mask A](#mask-a)
    2. [Mask B](#mask-b)
    3. [Mask C](#mask-c)
    4. [Mask D](#mask-d)
    5. [Mask E](#mask-e)
    6. [Mask F](#mask-f)


### Visualize images
Display and get familiar with the images.

In [ ]:
%use sos
# Declare image_list for ImageViewer
image_list = [bikesgray, camera, spots]

imgs_viewer = viewer(image_list, widgets=True, hist=True)

(edge-detection)=
# 1. Edge detection (3 points)
[Back to table of contents](#ToC_1_Filtering)

In this part of the lab, we will review one of the most basic image processing operation, edge detection. Moreover, we will use this filter to review the concepts of convolution and correlation. As you might remember from the course, edge detection algorithms try to detect sharp intensity differences between neighbouring pixels. A common vertical-edge detector (detecting changes in the $x$ direction) is the Sobel filter, represented by the impulse response

$$h[m,n] = 
\begin{bmatrix}
    -1 & 0 & 1 \\
    -2 & 0 & 2 \\
    -1 & 0 & 1
\end{bmatrix}\,.$$

If pixels surrounding a given pixel position all have a similar value, the output at that pixel will have a value close to zero after applying the mask. On the other hand, if there is a big difference in the values of the left and right columns, this likely indicates some kind of vertical edge and consequently the output at that location will have either a highly positive or highly negative intensity.  

As you saw in the lecture, there are filters that you can implement in a separable way, and filters that you cannot. Since the separable way is the most efficient one, you should develop the ability to recognize filters that are separable and implement them as such. Go through the next sections to implement both methods!

(separable-implementation)=
## 1.A. Separable implementation (2 points)
[Back to table of contents](#ToC_1_Filtering)

In this subsection, you are going to modify the function `filterSeparable`, that we provide to implement the separable version of the filter. This function is based on the function `filter1D` (also declared in the next cell), which takes as input a row/column of the image and a 1D mask and performs a 1D convolution. Recall that separable filters are divided into two 1D masks (`mx` and `my` in the code below). 

In the next cell, **for 1 point** complete the function `filterSeparable` by implementing the column filtering. The row filtering is provided and can be used as a reference.

In [ ]:
%use javascript

// function that performs a separable convolution on img using two 1D masks mx and my
function filterSeparable(img, mx, my){
    // initialize output Image, of the same shape as the input
    var output = new Image(img.shape());
    
    // iterate through every row 
    for(var y = 0; y < img.ny; y++){
        // extract row
        var row = img.getRow(y);
        // apply filter
        var new_row = filter1D(row, mx)
        // set column in output variable
        output.putRow(y, new_row);    
    }
    // iterate through every column
    for(var x = 0; x < img.nx; x++){
        // filter the columns
        
        // extract column, apply filter and set
        // YOUR CODE HERE
        
    }
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    // create the output image
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1);
        // declare a variable to store the values of the convolution. 
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        // set value in output array
        output.setPixel(x, 0, val);
    }
    return output
}

### 1D masks

Now, before testing your separable implementation, **for 1 point**, code the 1D masks `mask_x` and `mask_y` in the cell below that corresponds to the separable version of the vertical-edge detection filter presented [at the beginning of this section](#1.-Edge-detection-(3-points)).

In [ ]:
%use javascript
%put mask_x mask_y
// change the masks to the correct value 
var mask_x = [0, 0, 0];
var mask_y = [0, 0, 0];

// YOUR CODE HERE

// convert the masks to Image objects
var mask_x_img = new Image([mask_x]);
var mask_y_img = new Image([mask_y]);

Now, run the next cell for a quick sanity check on your function `filterSeparable` and on your $1$D masks. We will convolve an impulse with your $1$D masks, and the result should be exactly the filter that we presented [at the beginning of the section](#edge-detection).

In [ ]:
%use javascript
// again let's do a sanity check with an impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare correct 2D mask
var mask = new Image([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]);
// the result of a convolution with an impulse image should be the mask itself
console.log('Convolving by an impulse should return the original 2D mask.\nMask:\n' + mask.visualize() )
console.log('Result:\n' + filterSeparable(impulse, mask_x_img, mask_y_img).visualize(), '\n');

// First we check your masks with a low tolerance
if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask, tol=1e-12) == false){
    // Repeat with high tolerance
    if(filterSeparable(impulse, mask_x_img, mask_y_img).imageCompare(mask, tol=1e-4) == false){
    console.log('WARNING!!!\nSorry it looks like the separable filter or the 1D masks are not yet correct.');
    }else{
        console.log('WARNING!!!\nYour masks are almost correct, but the accuracy does not seem quite enough...');
}}else{
    console.log('The separable filter and the 1D masks have passed this sanity check!');
}

Now run the next cell to apply the function to three real images.

In [ ]:
%use javascript
%get bikesgray spots camera
// Apply function on images
var vedge_bike_s = filterSeparable(new Image(bikesgray), mask_x_img, mask_y_img).toArray();
var vedge_spots_s = filterSeparable(new Image(spots), mask_x_img, mask_y_img).toArray();
var vedge_camera_s = filterSeparable(new Image(camera), mask_x_img, mask_y_img).toArray();

Run the next one to visualize the results.

In [ ]:
%use sos
%get vedge_bike_s vedge_camera_s vedge_spots_s --from javascript
# Declare the image list
image_list_sep = [vedge_bike_s, vedge_camera_s, vedge_spots_s]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']

# Make sure that the images are numpy arrays and not lists
for i, image in enumerate(image_list_sep):
    image_list_sep[i] = np.array(image)

plt.close('all')
vedge_s_viewer = viewer(image_list_sep, title=title_list, widgets=True)

(non-separable-implementation)=
## 1.B. Nonseparable implementation (1 point)
[Back to table of contents](#ToC_1_Filtering)

In this section, we will implement a 2D convolution function for filters that are not separable. We will test it on the following nonseparable mask:

$$\begin{bmatrix}
        -3  & 1 & 3 \\
        -10 & 1 & 10 \\
        -3  & 1 & 3
    \end{bmatrix}.$$
In the next cell we provide such a function `filterNonSeparable` that takes as input `img` and `mask`. This function however confuses correlation with convolution. **For 1 point**, find the mistake and correct it in the cell below.

:::{hint}
* The two-dimensional convolution between two images $h$ and $f$ is given by
$$(h \ast f)[x,y] = \sum_{m \in \mathbb{Z}}\sum_{n \in \mathbb{Z}}f[m,n]h[x-m,y-n]$$
* You can find inspiration from the previous function `filter1D`.
:::

In [ ]:
%use javascript

// function that performs a convolution on the two input parameters img and mask
function filterNonSeparable(img, mask){
    // create output Image, of the same shape as the input
    var output = new Image(img.shape());
    // iterate through every pixel
    for(var x = 0; x < img.nx; x++){  
        for(var y = 0; y < img.ny; y++){
            // get neighbourhood with the size of mask of current position
            var neigh = img.getNbh(x, y, mask.nx, mask.ny)
            // perform convolution
            var val = 0;
            for(var k = 0; k < mask.nx; k++){
                for(var l = 0; l < mask.ny; l++){
                    val += neigh.getPixel(k, l) * mask.getPixel(k, l);
                }
            }
            // assign value on output image
            output.setPixel(x, y, val)
        }
    }
    return output
}

Run the next cell to perform a sanity check (convolution of the mask with an impulse image).

In [ ]:
%use javascript

// Sanity check with a simple 3x3 impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);
// declare mask
var mask = new Image([[-3, 1, 3],[-10, 1, 10],[-3, 1, 3]])
var nonsymmetric_mask = new Image([[1, 2, 3],[4, 5, 6],[7, 8, 9]]);
// check the result
console.log('Convolving by an impulse should return the unchanged mask.\nMask:\n' + mask.visualize() )
console.log('\n' + nonsymmetric_mask.visualize())
console.log('Symmetric Result:\n' + filterNonSeparable(impulse, mask).visualize());
console.log('Nonsymmetric Result:\n' + filterNonSeparable(impulse, nonsymmetric_mask).visualize());
if(filterNonSeparable(impulse, mask).imageCompare(mask) == false){
    console.log('\nWARNING!!!\nThe function `filterNonSeparable` still provides an incorrect output.\n')
}else{
    console.log('It seems that the function works with a symmetric mask.')
    if (filterNonSeparable(impulse, nonsymmetric_mask).imageCompare(nonsymmetric_mask) == false){
        console.log('\nWARNING!!!\nIt seems that the function does not work with a nonsymmetric mask however.\n')
    }
    else{
        console.log('It seems that the function also works with a nonsymmetric mask')
    }
}

### Applying the function

Run the next cell to apply the `filterNonSeparable` function with the mask given above to the images `bikesgray`, `camera` and `spots`. 

In [ ]:
%use javascript
%get bikesgray camera spots

// apply nonseparable filter
var vedge_bike_nons   = filterNonSeparable(new Image(bikesgray), mask).toArray();
var vedge_spots_nons  = filterNonSeparable(new Image(spots),     mask).toArray();
var vedge_camera_nons = filterNonSeparable(new Image(camera),    mask).toArray();

### Visualize the results
In the next cell we will visualize the results of edge detection. 

In [ ]:
%use sos
%get vedge_bike_nons vedge_camera_nons vedge_spots_nons --from javascript

# Declare parameters for viewer
image_list_nons = [vedge_bike_nons, vedge_camera_nons, vedge_spots_nons]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']

# Make sure that the images are numpy arrays and not lists
for image in range(len(image_list_nons)):
    image_list_nons[image] = np.array(image_list_nons[image])

vedge_nons_viewer = viewer(image_list_nons, title=title_list, widgets=True)

(comparison-separable-vs-non-separable)=
## 1.C. Comparison: Separable vs Nonseparable
[Back to table of contents](#ToC_1_Filtering)

Are the results of the the separable and nonseparable implementations of a separable filter the same? If both your implementations are correct, the difference should be negligible. 

Run the next cell to apply your nonseparable implementation to the images `bikesgray`, `camera` and `spots`, using the Sobel filter at the [beginning of this section](#edge-detection). 

In [ ]:
%use javascript
%get bikesgray camera spots

var mask = new Image([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]]);

// apply Non separable filter
var vedge_bike_nons   = filterNonSeparable(new Image(bikesgray), mask).toArray();
var vedge_spots_nons  = filterNonSeparable(new Image(spots),     mask).toArray();
var vedge_camera_nons = filterNonSeparable(new Image(camera),    mask).toArray();

Now, go ahead and do some exploration! Try to plot some of the images next to each other, look at the two histograms, zoom into different areas of the three images, and see if they are truly equal. There are several tests you can do to measure the equality of both implementations. You can even take the difference between the two images and visualize it (`image1 - image2`), or you can use the different functions provided by NumPy to compare arrays (see [numpy.testing.assert_array_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_equal.html) and [numpy.testing.assert_array_almost_equal](https://numpy.org/doc/stable/reference/generated/numpy.testing.assert_array_almost_equal.html)).


Use the next blank cell to do any necessary exploration! Note that we have gotten the filtered images from the JavaScript kernel, put them in a list called `image_list_nons` and ensured that they are NumPy arrays.

In [ ]:
%use sos
%get vedge_bike_nons vedge_camera_nons vedge_spots_nons --from javascript

# Declare parameters for viewer
image_list_nons = [vedge_bike_nons, vedge_camera_nons, vedge_spots_nons]
title_list = ['Bike - Vertical edges', 'Camera - Vertical edges', 'Spots - Vertical edges']

# Make sure that the images are numpy arrays and not lists
for image in range(len(image_list_nons)):
    image_list_nons[image] = np.array(image_list_nons[image])

# YOUR CODE HERE

(edge-detection-in-python)=
## 1.D. Edge detection in Python
[Back to table of contents](#ToC_1_Filtering)

In Python, the two functions `filterNonSeparable` and `filterSeparable` you implemented correspond to [`convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html) and [`convolve1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve1d.html) from the [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html) library. We will compare your implementation with them. Read the documentation of `convolve` and `convolve1d` and pay attention to the usage of these two functions in the next cell as you will use these functions in the next section.

If you did everything correctly in the [section on nonseparable implementation](#non-separable-implementation) and the [section on separable implementation](#separable-implementation), the next cell will display a winning message. Otherwise, it will plot your images with the regions that differ from the correct answer highlighted in red, this should give you a clue on where you might have gone wrong. For this purpose we will use the function [`np.allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html), a standard NumPy comparison tool. Furthermore, you can uncomment the final line to visualize the four edge detections we have so far.

:::{note}
In this lab we have been using `float64` ($64$-bit floating point) as the `dtype` (which stands for *data type*) of the images (look at the [Imports section](#imports). This `dtype` is the most versatile, and since the `dtype` of the output array is the same (by default) as the dtype of the input array, we don't really need to mind the `output` parameter right now. 

However, it is also very common to store images and work with them as `np.uint8` (8-bit unsigned integer, which as you should know spans the range $[0, 255]$). Nonetheless we know that the gradient can take large positive and negative values, so an [unsigned integer](https://en.wikipedia.org/wiki/Integer_(computer_science)) is not the right type. If you don't realize why, do a quick experiment: declare a NumPy Array of length one, of data type `'uint8'`, and assign a value outside of the range $[0, 255]$ (e.g. `np.array([300], dtype = 'uint8')`). Perform a few operations and see what happens.

If at some point you need to work with integers (e.g. to save memory or to increase performance), because we know that the result will still be an integer -the mask only contains integers- you can choose `np.int32` (32-bit signed integers). 
This is something that you will have to keep in mind constantly when working with images, and that can be the source of many bugs.
:::

In [ ]:
%use sos 
# Define the 2D mask
mask = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
# Apply nonseparable implementation with 2D mask
vedge_ndi = ndi.convolve(bikesgray, mask, output=np.int32)

# Apply separable implementation - we use mask_x and mask_y you defined in the exercise before
# filter the columns
vedge_ndi_s = ndi.convolve1d(bikesgray, mask_y, axis=0, output=np.float64)
# filter the rows
vedge_ndi_s = ndi.convolve1d(vedge_ndi_s, mask_x, axis=1, output=np.float64)

# Visualize if necessary
plt.close('all')
if np.allclose(vedge_ndi, vedge_bike_nons):
    print('Your nonseparable implementation is correct!')
else :
    print('Review your nonseparable JS implementation!')
    viewer([np.array(vedge_bike_nons), vedge_ndi], title=['JS nonseparable', 'Python nonseparable'], compare=True)

if np.allclose(vedge_ndi_s, vedge_bike_s):
    print('Congratulations!\nYour separable filter is correct!')
else :
    print('Review your separable JS implementation!\
           \nThe problem could be either on your 1D masks, or on the implementation itself. The previous sanity checks and the red areas should guide you.')
    viewer([np.array(vedge_bike_s), vedge_ndi_s], title=['JS separable', 'Python separable'], compare=True)

# Uncomment the following line to visualize the four edge detections we have done so far
# vedge_viewer = viewer([np.array(vedge_bike_nons), vedge_ndi, np.array(vedge_bike_s), vedge_ndi_s], title=['JS nonseparable', 'Python nonseparable', 'JS separable', 'Python separable'], subplots=(2,2))

(implementation-and-classification-of-digital-filters)=
# 2. Implementation and classification of digital filters (6 points)
[Back to table of contents](#ToC_1_Filtering)

In this exercise, we will practice the implementation and classification of masks or digital filters. Moreover, you will practice separable and nonseparable implementations in Python. You will implement 6 functions in Python that correspon to 6 different digital filters. Each of them takes an input image and outputs the filtered version of it. You need to
* hard code the mask inside the function,
* apply the filtering using either `convolve` and `convolve1d`, depending on if the filter is separable.

**If a mask is separable and you don't implement it as such, you will not get the point. You should always use `convolve` for nonseparable filters and `convolve1d` for separable ones.**
Implementing each function is worth **0.5** points, classifing their visual effect is worth **0.5** points. 

You can define 1D and 2D filters as 1D and 2D numpy arrays, e.g.
```python
mask = np.array([1, 2])
```
```python
mask = np.array([[ 1,  2], 
                 [ 6,  7]])
```

:::{note}Note on the separability of filters
Some filters, in particular simple edge detector filters can intuitively be recognized as separable. Furthermore, their $1\mathrm{D}$ masks can also be intuitively found. From these $1\mathrm{D}$ masks, a simple test consists in taking the outer product between them, which should result in the original filter. However, there is a more systematic approach.

As you should know, any matrix $\mathbf{\mathrm{M}}\in\mathbb{R}^{m\times n}$ can be factorized into its components $\mathbf{\mathrm{USV^{T}}}$ by Singular Value Decomposition, where $\mathbf{\mathrm{U}}\in\mathbb{R}^{m\times m}$, $\mathbf{\mathrm{S}}\in\mathbb{R}^{\min(m, n)\times \min(m, n)}$ corresponds to a diagonal matrix with the singular values of $\mathbf{\mathrm{M}}$, and $\mathbf{\mathrm{V}}\in\mathbb{R}^{n\times n}$. It follows from the definition of the matrix product that if only one singular value is larger than zero, the filter is separable.

Nonetheless, for simple filters, we still recommend you to use the *intuitive* filters. If you want to extract the filters using SVD, you can use the [`svd`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html) function of the NumPy Linear Algebra package. We leave you the task of extracting the actual $1\mathrm{D}$ masks from it.
:::

(mask-a)=
## 2.A. Mask A
[Back to table of contents](#ToC_1_Filtering)

```python3
    [ 0,  0,  1,  0,  0]
    [ 0,  1,  2,  1,  0]
A = [ 1,  2,-16,  2,  1]
    [ 0,  1,  2,  1,  0]
    [ 0,  0,  1,  0,  0]
```

In [ ]:
%use sos

# Function that filters img with the mask A
def filter_A(img):
    output = img.copy()
    
    # Filter the image with mask A    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_A` to the image `bikesgray`. 

In [ ]:
%use sos
# Apply the function
bikesgray_A = filter_A(bikesgray)
# Visualize the result
plt.close('all')
vis_A = viewer([bikesgray, bikesgray_A], title=['Original', 'Convolved with filter A'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. reveal edges and details in any directions (by a Laplacian filter), or
10. it has not effect.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

(mask-b)=
## 2.B. Mask B
[Back to table of contents](#ToC_1_Filtering)

```python3
    [ 0,  1,  2,  1,  0]
    [ 0,  1,  2,  1,  0]
B = [ 0,  1,  2,  1,  0]
    [ 0,  1,  2,  1,  0]
    [ 0,  1,  2,  1,  0]
```

In [ ]:
%use sos

# Function that filters img with the mask B
def filter_B(img):
    output = img.copy()
    
    # Filter the image with mask B    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_B` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_B = filter_B(bikesgray)
# Visualize the result
plt.close('all')
vis_B = viewer([bikesgray, bikesgray_B], title=['Original', 'Filtered with mask B'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. reveal edges and details in any directions (by a Laplacian filter), or
10. it has not effect.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

(mask-c)=
## 2.C. Mask C
[Back to table of contents](#ToC_1_Filtering)

```python3
    [-1,  -1,  -1,  -1,  -1]
    [-2,  -2,  -2,  -2,  -2]
C = [ 0,   0,   0,   0,   0]
    [ 2,   2,   2,   2,   2]
    [ 1,   1,   1,   1,   1]
```

In [ ]:
%use sos

# Function that filters img with the mask C
def filter_C(img):
    output = img.copy()
    
    # Filter the image with mask C
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_C` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_C = filter_C(bikesgray)
# Visualize the result
plt.close('all')
vis_C = viewer([bikesgray, bikesgray_C], title=['Original', 'Filtered with mask C'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. reveal edges and details in any directions (by a Laplacian filter), or
10. it has not effect.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

(mask-d)=
## 2.D. Mask D
[Back to table of contents](#ToC_1_Filtering)

```python3
    [-1,  -4,  -6,  -4, -1]
    [-4, -16, -24, -16, -4]
D = [-6, -24, 476, -24, -6]
    [-4, -16, -24, -16, -4]
    [-1,  -4,  -6,  -4, -1]
```

In [ ]:
%use sos

# Function that filters img with the mask D
def filter_D(img):
    output = img.copy()
    
    # Filter the image with mask D
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_D` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_D = filter_D(bikesgray)
# Visualize the result
plt.close('all')
vis_D = viewer([bikesgray, bikesgray_D], title=['Original', 'Filtered with mask D'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. reveal edges and details in any directions (by a Laplacian filter), or
10. it has not effect.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

(mask-e)=
## 2.E. Mask E
[Back to table of contents](#ToC_1_Filtering)

```python3
    [1, 1, 1, 1, 1]
    [1, 1, 1, 1, 1]
E = [1, 1, 1, 1, 1]
    [1, 1, 1, 1, 1]
    [1, 1, 1, 1, 1]
```

In [ ]:
%use sos

# Function that filters img with the mask E
def filter_E(img):
    output = img.copy()
    
    # Filter the image with mask E
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_E` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_E = filter_E(bikesgray)
# Visualize the result
plt.close('all')
vis_E = viewer([bikesgray, bikesgray_E], title=['Original', 'Filtered with mask E'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. reveal edges and details in any directions (by a Laplacian filter), or
10. it has not effect.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

(mask-f)=
## 2.F. Mask F
[Back to table of contents](#ToC_1_Filtering)

```python3
    [ 0, -1, -1,  0,  0]
    [-1, -1,  0,  0,  0]
F = [-1,  0,  0,  0,  1]
    [ 0,  0,  0,  1,  1]
    [ 0,  0,  1,  1,  0]
```

In [ ]:
%use sos

# Function that filters img with the mask F
def filter_F(img):
    output = img.copy()
    
    # Filter the image with mask F
    
    # YOUR CODE HERE
    
    return output

Run the next cell to visualize the result of applying `filter_F` to the image `bikesgray`.

In [ ]:
%use sos
# Apply the function
bikesgray_F = filter_F(bikesgray)
# Visualize the result
plt.close('all')
vis_F = viewer([bikesgray, bikesgray_F], title=['Original', 'Filtered with mask F'], subplots=(1,2))

**For 0.5 points** answer the following MCQ:

* Which of the following keywords best describes the visual effect of this filter?

1. isotropic blurring, or
2. vertical blurring, or
3. horizontal blurring, or
4. diagonal blurring, or
5. sharpening, or
6. vertical-edge detection, or
7. horizontal-edge detection, or
8. diagonal edge detection, or
9. reveal edges and details in any directions (by a Laplacian filter), or
10. it has not effect.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in list(range(1,11)):
    print('WARNING!\nAnswer one of 1, 2, 3, 4, 5, 6, 7, 8, 9 or 10.')

Congratulations on finishing the first part of the Filtering lab!

:::{attention}
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to [Moodle](https://moodle.epfl.ch/mod/assign/view.php?id=1157357), in a zip file with the other notebook of this lab.

* Keep the name of the notebook as: *1_Filtering.ipynb*,
* Name the zip file: *Filtering_lab.zip*.
:::